In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:27<00:00, 41.56s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Refurb Tineco Floor ONE S6 Cordless Wet Dry Vacuum for $220 + free shipping w/ Prime\nDetails: You' pay $499 for a new one at Amazon. A 1-year Allstate warranty applies. Buy Now at eBay\nFeatures: Cleans wet and dry messes Advanced self-cleaning feature Dual-sided edge cleaning iLoop smart sensor technology Cordless convenience\nURL: https://www.dealnews.com/Refurb-Tineco-Floor-ONE-S6-Cordless-Wet-Dry-Vacuum-for-220-free-shipping-w-Prime/21746705.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Unlocked Samsung Galaxy S22 Ultra 128GB 5G Android Phone for $293 + free shipping
Details: It's the best price we've seen for this S22 Ultra model. You'd pay $47 more for a refurb today at Amazon. It comes with a 1-year Allstate warranty. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/products/Samsung/Samsung-Galaxy-S22-Ultra-128-GB-5-G-Android-Phon

In [26]:
# Initialize and constants

load_dotenv(override=True)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

client = OpenAI(
    api_key=GITHUB_TOKEN,
    base_url="https://models.github.ai/inference",
)

def get_recommendations():
    completion = client.beta.chat.completions.parse(
        model="openai/gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [28]:
result = get_recommendations()

In [29]:
len(result.deals)

5

In [30]:
result.deals[1]

Deal(product_description='The bObsweep Dustin Wi-Fi Self-Emptying Robot Vacuum & Mop offers a high-tech cleaning experience, featuring a 70-day self-empty container, voice control via Alexa or Google Assistant, and customizable mapping capabilities. With three adjustable suction settings, it efficiently tackles various floor types, making it a versatile cleaning solution for busy households. The vacuum’s automated functions and smart technology simplify the cleaning process dramatically.', price=750.0, url='https://www.dealnews.com/products/bObsweep/bObsweep-Dustin-Wi-Fi-Self-Emptying-Robot-Vacuum-Mop/462410.html?iref=rss-f1912')

In [1]:
from agents.scanner_agent import ScannerAgent

In [5]:
agent = ScannerAgent()
result = agent.scan()

In [6]:
result

DealSelection(deals=[Deal(product_description='The Refurb Unlocked Samsung Galaxy S22 Ultra is a top-of-the-line 5G Android phone featuring 128GB of storage. This model boasts impressive capabilities including a stunning display, advanced camera systems, and the flexibility of being unlocked for any carrier. It also comes with a 1-year warranty, ensuring quality and support for your purchase. Perfect for users looking to upgrade to a premium smartphone experience.', price=293.0, url='https://www.dealnews.com/products/Samsung/Samsung-Galaxy-S22-Ultra-128-GB-5-G-Android-Phone/477144.html?iref=rss-c142'), Deal(product_description='The Refurb Unlocked Apple iPhone 15 Plus offers an incredible user experience with 128GB of storage. It features state-of-the-art technology, a high-resolution camera, and seamless integration with iOS and your other Apple devices. This refurbished model comes with a 90-day warranty, providing peace of mind with your purchase. Ideal for Apple enthusiasts looking